## TODO

Look at dataset, list by size and pick top 100 

In [1]:
import json
from pprint import pprint
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.patches as mpatches
import matplotlib
matplotlib.style.use('ggplot')
#ols models
import statsmodels.formula.api as sm
from patsy import dmatrices
import re
import scipy as sp
from tools import print_columns
import csv

Possible screening rules 
- proximity to homeowner, need population per zip code in San Jose. Simple assume that just density per tile. One tile = one zip code. 
- offered savings, need to convert price per watt into savings 
- customer rating 
- 

For each in top 100 create customer rating  - scrape yelp  

pick 1 installer from narrowed down via screening rules, this installer is offering 1 project - compare to none option and pick it. Depends on the 

interaction - depends on the number of installed projects - goes by size
distribution of locations - from NEM


warranty - could not get distribution, assume basically it from bla

total project time - ? need to think
will have from dataset some version of a time 


equipment technology - will have from dataset, but need to actually transform data - will need a lot of help. 
Alternative is assign by size with some distribution.

from panels on the market that are used by installers - get type of installer - panel efficiency, panel visibility

1 installer is offering 2? types of systems - look into actual data to see what types they are offering 

failures in first years - tie into inverter type. 
Assume basic failure rate. Check with actual data that might get from the survey. 
Question about amount of maintenance that need

Top 100 for each have Rating, InteractionType, EquipmentType, TotalProjectTime, Warranty

Generate 100 agents in Python with customization and import list of parameters into c++

In [2]:
sei = {'EParamTypes::SEIRating':3.0, 
       'EParamTypes::SEIInteractionType':0.0,
       'EParamTypes::SEIEquipmentType':1.0, 
       'EParamTypes::SEILeadInProjectTime':10.0, 
       'EParamTypes::SEIWarranty':5*52}

In [3]:
seis = []
#create test sei 
for i in range(3):
    sei['EParamTypes::SEIRating'] = 3.0 + i
    seis.append(sei.copy())
    

In [4]:
with open('sei-params.json', 'w') as outfile:
    json.dump(seis, outfile)

In [ ]:
#calculator for CO2
#https://www.epa.gov/energy/greenhouse-gas-equivalencies-calculator
    
    
1000 - 0.665